# Phase 1: Best Configuration Selection (Local, Google Colab & Kaggle)

This notebook automates the selection of the best model configuration from MLflow
based on metrics and benchmarking results, then performs final training and model conversion.


## Workflow

**Prerequisites**: Run `01_orchestrate_training_colab.ipynb` first to:
- Train models via HPO
- Run benchmarking on best trials (using `evaluation.benchmarking.benchmark_best_trials`)

Then this notebook:

1. **Best Model Selection**: Query MLflow benchmark runs, join to training runs via grouping tags (`code.study_key_hash`, `code.trial_key_hash`), select best using normalized composite scoring
2. **Artifact Acquisition**: Download the best model's checkpoint using fallback strategy (local disk → drive restore → MLflow download)
3. **Final Training**: Optionally retrain with best config on full dataset (if not already final training)
4. **Model Conversion**: Convert the final model to ONNX format using canonical path structure


## Important

- This notebook **executes on Local, Google Colab, or Kaggle** (not on Azure ML compute)
- Requires MLflow tracking to be set up (Azure ML workspace or local SQLite)
- All computation happens on the platform's GPU (if available) or CPU
- **Storage & Persistence**:
  - **Local**: Outputs saved to `outputs/` directory in repository root
  - **Google Colab**: Checkpoints are automatically saved to Google Drive for persistence across sessions
  - **Kaggle**: Outputs in `/kaggle/working/` are automatically persisted - no manual backup needed
- The notebook must be **re-runnable end-to-end**
- Uses the dataset path specified in the data config (from `config/data/*.yaml`), typically pointing to a local folder included in the repository
- **Session Management**:
  - **Local**: No session limits, outputs persist in repository
  - **Colab**: Sessions timeout after 12-24 hours (depending on Colab plan). Checkpoints are saved to Drive automatically.
  - **Kaggle**: Sessions have time limits based on your plan. All outputs are automatically saved.


## Step 1: Environment Detection

The notebook automatically detects the execution environment (local, Google Colab, or Kaggle) and adapts its behavior accordingly.


In [1]:
import sys
from pathlib import Path

# Bootstrap: Find repository root and add src/ to Python path
# This must happen before importing from common or infrastructure
def find_repo_root() -> Path:
    """Find repository root by searching for config/ and src/ directories."""
    current_dir = Path.cwd()
    # Check current directory first
    if (current_dir / "config").exists() and (current_dir / "src").exists():
        return current_dir
    # Search up the directory tree
    for parent in current_dir.parents:
        if (parent / "config").exists() and (parent / "src").exists():
            return parent
    raise ValueError(f"Could not find repository root. Searched from: {current_dir}")

# Find repo root and add src to path
ROOT_DIR = find_repo_root()
SRC_DIR = ROOT_DIR / "src"
if str(SRC_DIR) not in sys.path:
    sys.path.insert(0, str(SRC_DIR))

# Now we can import from common
from common.shared.notebook_setup import detect_notebook_environment

# Detect execution environment
env = detect_notebook_environment()
PLATFORM = env.platform
IN_COLAB = env.is_colab
IN_KAGGLE = env.is_kaggle
IS_LOCAL = env.is_local
BASE_DIR = env.base_dir
BACKUP_ENABLED = env.backup_enabled

print(f"✓ Detected environment: {PLATFORM.upper()}")
print(f"Platform: {PLATFORM}")
print(f"Base directory: {BASE_DIR if BASE_DIR else 'Current working directory'}")
print(f"Backup enabled: {BACKUP_ENABLED}")


/opt/conda/envs/resume-ner-training/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✓ Detected environment: LOCAL
Platform: local
Base directory: Current working directory
Backup enabled: False


### Install Required Packages

Install required packages based on the execution environment.


In [2]:
# Install required packages
if IS_LOCAL:
    print("For local environment, please:")
    print("1. Create conda environment: conda env create -f config/environment/conda.yaml")
    print("2. Activate: conda activate resume-ner-training")
    print("3. Restart kernel after activation")
    print("\nIf you've already done this, you can continue to the next cell.")
    print("\nInstalling Azure ML SDK (required for imports)...")
    # Install Azure ML packages even for local (in case conda env not activated)
    %pip install "azure-ai-ml>=1.0.0" --quiet
    %pip install "azure-identity>=1.12.0" --quiet
    %pip install azureml-defaults --quiet
    %pip install azureml-mlflow --quiet
else:
    # Core ML libraries
    %pip install "transformers>=4.35.0,<5.0.0" --quiet
    %pip install "safetensors>=0.4.0" --quiet
    %pip install "datasets>=2.12.0" --quiet

    # ML utilities
    %pip install "numpy>=1.24.0,<2.0.0" --quiet
    %pip install "pandas>=2.0.0" --quiet
    %pip install "scikit-learn>=1.3.0" --quiet

    # Utilities
    %pip install "pyyaml>=6.0" --quiet
    %pip install "tqdm>=4.65.0" --quiet
    %pip install "seqeval>=1.2.2" --quiet
    %pip install "sentencepiece>=0.1.99" --quiet

    # Experiment tracking
    %pip install mlflow --quiet
    %pip install optuna --quiet

    # Azure ML SDK (required for orchestration imports)
    %pip install "azure-ai-ml>=1.0.0" --quiet
    %pip install "azure-identity>=1.12.0" --quiet
    %pip install azureml-defaults --quiet
    %pip install azureml-mlflow --quiet

    # ONNX support
    %pip install onnxruntime --quiet
    %pip install "onnx>=1.16.0" --quiet
    %pip install "onnxscript>=0.1.0" --quiet

    print("✓ All dependencies installed")


For local environment, please:
1. Create conda environment: conda env create -f config/environment/conda.yaml
2. Activate: conda activate resume-ner-training
3. Restart kernel after activation

If you've already done this, you can continue to the next cell.

Installing Azure ML SDK (required for imports)...
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


## Step 2: Repository Setup

**Note**: Repository setup is only needed for Colab/Kaggle environments. Local environments should already have the repository cloned.


In [3]:
# Repository setup - only needed for Colab/Kaggle
if not IS_LOCAL:
    if IN_KAGGLE:
        !git clone -b gg_final_training_2 https://github.com/longdang193/resume-ner-azureml.git /kaggle/working/resume-ner-azureml
    elif IN_COLAB:
        !git clone -b gg_final_training_2 https://github.com/longdang193/resume-ner-azureml.git /content/resume-ner-azureml
else:
    print("✓ Local environment detected - detecting repository root...")

# Set up paths using extracted utility
from common.shared.notebook_setup import setup_notebook_paths

paths = setup_notebook_paths(add_src_to_path=True)
ROOT_DIR = paths.root_dir
CONFIG_DIR = paths.config_dir
SRC_DIR = paths.src_dir

print(f"✓ Repository: {ROOT_DIR} (config={CONFIG_DIR.name}, src={SRC_DIR.name})")
print("✓ Repository structure verified")


✓ Local environment detected - detecting repository root...
✓ Repository: /workspaces/resume-ner-azureml (config=config, src=src)
✓ Repository structure verified


## Step 3: Load Configuration

Load experiment configuration and define experiment naming convention.


In [5]:
from infrastructure.config.loader import load_experiment_config
from common.constants import EXPERIMENT_NAME
from common.shared.yaml_utils import load_yaml
from infrastructure.naming.mlflow.tags_registry import load_tags_registry

# Load experiment config
experiment_config = load_experiment_config(CONFIG_DIR, EXPERIMENT_NAME)

# Load best model selection configs
tags_config = load_tags_registry(CONFIG_DIR)
selection_config = load_yaml(CONFIG_DIR / "best_model_selection.yaml")
conversion_config = load_yaml(CONFIG_DIR / "conversion.yaml")
acquisition_config = load_yaml(CONFIG_DIR / "artifact_acquisition.yaml")
benchmark_config = load_yaml(CONFIG_DIR / "benchmark.yaml")

print(f"✓ Loaded configs: experiment={experiment_config.name}, tags, selection, conversion, acquisition, benchmark")

# Define experiment names (discovery happens after MLflow setup in Cell 4)
experiment_name = experiment_config.name
benchmark_experiment_name = f"{experiment_name}-benchmark"
training_experiment_name = f"{experiment_name}-training"  # For final training runs
conversion_experiment_name = f"{experiment_name}-conversion"

print(f"✓ Experiment names: benchmark={benchmark_experiment_name}, training={training_experiment_name}, conversion={conversion_experiment_name}")


✓ Loaded configs: experiment=resume_ner_baseline, tags, selection, conversion, acquisition, benchmark
✓ Experiment names: benchmark=resume_ner_baseline-benchmark, training=resume_ner_baseline-training, conversion=resume_ner_baseline-conversion


## Step 4: Setup MLflow

Setup MLflow tracking with fallback to local if Azure ML is unavailable.


In [6]:
# Check if azureml.mlflow is available
try:
    import azureml.mlflow  # noqa: F401
    print("✓ azureml.mlflow is available - Azure ML tracking will be used if configured")
except ImportError:
    print("⚠ azureml.mlflow is not available - will fallback to local SQLite tracking")
    print("  To use Azure ML tracking, install: pip install azureml-mlflow")
    print("  Then restart the kernel and re-run this cell")

from common.shared.mlflow_setup import setup_mlflow_from_config
import mlflow

# Setup MLflow tracking (use training experiment for setup - actual queries use discovered experiments)
tracking_uri = setup_mlflow_from_config(
    experiment_name=training_experiment_name,
    config_dir=CONFIG_DIR,
    fallback_to_local=True,
)

print(f"✓ MLflow tracking URI: {tracking_uri}")
print(f"✓ MLflow experiment: {training_experiment_name}")

# Discover HPO and benchmark experiments from MLflow (after setup)
# NOTE: This cell is the SINGLE SOURCE OF TRUTH for hpo_experiments and benchmark_experiment
# These variables are reused in:
#   - Cell 16 (Step 6: Benchmarking) - uses hpo_experiments and benchmark_experiment
#   - Cell 18 (Step 7: Best Model Selection) - uses hpo_experiments and benchmark_experiment
# Do not rebuild these variables elsewhere - always reference them from this cell.
from mlflow.tracking import MlflowClient
from evaluation.selection.experiment_discovery import discover_all_experiments

mlflow_client = MlflowClient()
experiments = discover_all_experiments(
    experiment_name=experiment_name,
    mlflow_client=mlflow_client,
    create_benchmark_if_missing=False,  # Don't auto-create, let benchmarking step handle it
)

hpo_experiments = experiments["hpo_experiments"]
benchmark_experiment = experiments["benchmark_experiment"]

hpo_backbones = ", ".join(hpo_experiments.keys())
print(f"✓ Experiments: {len(hpo_experiments)} HPO ({hpo_backbones}), benchmark={'found' if benchmark_experiment else 'not found'}, training={training_experiment_name}, conversion={conversion_experiment_name}")


2026-01-15 11:18:09,265 - common.shared.mlflow_setup - INFO - Azure ML enabled in config, attempting to connect...
2026-01-15 11:18:09,268 - common.shared.mlflow_setup - WARNING - [DEBUG] Initial env check - subscription_id: False, resource_group: False, client_id: False, client_secret: False, tenant_id: False
2026-01-15 11:18:09,268 - common.shared.mlflow_setup - INFO - Attempting to load credentials from config.env at: /workspaces/resume-ner-azureml/config.env
2026-01-15 11:18:09,269 - common.shared.mlflow_setup - INFO - Loading credentials from /workspaces/resume-ner-azureml/config.env
2026-01-15 11:18:09,273 - common.shared.mlflow_setup - INFO - Loaded subscription/resource group from config.env
2026-01-15 11:18:09,279 - common.shared.mlflow_setup - INFO - Loaded service principal credentials from config.env
2026-01-15 11:18:09,283 - common.shared.mlflow_setup - WARNING - [DEBUG] Platform detected: local
2026-01-15 11:18:09,285 - common.shared.mlflow_setup - WARNING - [DEBUG] Servi

✓ azureml.mlflow is available - Azure ML tracking will be used if configured


Class DeploymentTemplateOperations: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
2026-01-15 11:18:09,582 - common.shared.mlflow_setup - INFO - Successfully connected to Azure ML workspace: resume-ner-ws
2026-01-15 11:18:10,591 - common.shared.mlflow_setup - INFO - Using Azure ML workspace tracking
2026-01-15 11:18:10,671 - evaluation.selection.experiment_discovery - INFO - Discovered 2 HPO experiment(s) for resume_ner_baseline
2026-01-15 11:18:10,757 - evaluation.selection.experiment_discovery - INFO - Found benchmark experiment: resume_ner_baseline-benchmark


✓ MLflow tracking URI: azureml://germanywestcentral.api.azureml.ms/mlflow/v2.0/subscriptions/50c06ef8-627b-46d5-b779-d07c9b398f75/resourceGroups/resume_ner_2026-01-02-16-47-05/providers/Microsoft.MachineLearningServices/workspaces/resume-ner-ws
✓ MLflow experiment: resume_ner_baseline-training
✓ Experiments: 2 HPO (distilbert, distilroberta), benchmark=found, training=resume_ner_baseline-training, conversion=resume_ner_baseline-conversion


## Step 5: Drive Backup Setup (Colab Only)

Setup Google Drive backup/restore for Colab environments.


In [7]:
from pathlib import Path

# Fix numpy/pandas compatibility before importing orchestration modules
try:
    from infrastructure.storage.drive import create_colab_store
except (ValueError, ImportError) as e:
    if "numpy.dtype size changed" in str(e) or "numpy" in str(e).lower():
        print("⚠ Numpy/pandas compatibility issue detected. Fixing...")
        import subprocess
        import sys
        subprocess.check_call([sys.executable, "-m", "pip", "install", "--upgrade", "--force-reinstall", "--no-cache-dir", "numpy>=1.24.0,<2.0.0", "pandas>=2.0.0", "--quiet"])
        print("✓ Numpy/pandas reinstalled. Please restart the kernel and re-run this cell.")
        raise RuntimeError("Please restart kernel after numpy/pandas fix")
    else:
        raise

# Mount Google Drive and create backup store (Colab only - Kaggle doesn't need this)
DRIVE_BACKUP_DIR = None
drive_store = None
restore_from_drive = None

if IN_COLAB:
    drive_store = create_colab_store(ROOT_DIR, CONFIG_DIR)
    if drive_store:
        BACKUP_ENABLED = True
        DRIVE_BACKUP_DIR = drive_store.backup_root
        # Create restore function wrapper
        def restore_from_drive(local_path: Path, is_directory: bool = False) -> bool:
            """Restore file/directory from Drive backup."""
            try:
                expect = "dir" if is_directory else "file"
                result = drive_store.restore(local_path, expect=expect)
                return result.ok
            except Exception as e:
                print(f"⚠ Drive restore failed: {e}")
                return False
        print(f"✓ Google Drive mounted")
        print(f"✓ Backup base directory: {DRIVE_BACKUP_DIR}")
        print(f"\nNote: All outputs/ will be mirrored to: {DRIVE_BACKUP_DIR / 'outputs'}")
    else:
        BACKUP_ENABLED = False
        print("⚠ Warning: Could not mount Google Drive. Backup to Google Drive will be disabled.")
elif IN_KAGGLE:
    print("✓ Kaggle environment detected - outputs are automatically persisted (no Drive mount needed)")
    BACKUP_ENABLED = False
else:
    # Local environment
    print("✓ Local environment detected - outputs will be saved to repository (no Drive backup needed)")
    BACKUP_ENABLED = False


✓ Local environment detected - outputs will be saved to repository (no Drive backup needed)


## Step 6: Optional - Run Benchmarking on Champions

**Optional Step**: If you haven't run benchmarking in `01_orchestrate_training_colab.ipynb`, you can run it here before selecting the best model. This step will:
1. Select champions (best trials) from HPO runs using Phase 2 selection logic
2. Run benchmarking on each champion to measure inference performance
3. Save benchmark results to MLflow for use in Step 7

**Note**: If benchmark runs already exist in MLflow, you can skip this step and proceed directly to Step 7.


In [8]:
# Optional: Run benchmarking on champions if not already done
# Skip this cell if benchmark runs already exist in MLflow

RUN_BENCHMARKING = True  # Set to True to run benchmarking

if RUN_BENCHMARKING:
    from evaluation.selection.workflows import run_benchmarking_workflow
    from orchestration.jobs.tracking.mlflow_tracker import MLflowBenchmarkTracker
    from infrastructure.config.loader import load_all_configs
    
    # Load all configs
    configs = load_all_configs(experiment_config)
    data_config = configs.get("data", {})
    hpo_config = configs.get("hpo", {})
    
    # Use benchmark_experiment from Cell 12 (single source of truth) if available
    benchmark_experiment_name = f"{experiment_name}-benchmark"
    if "benchmark_experiment" not in globals() or benchmark_experiment is None:
        from evaluation.selection.experiment_discovery import discover_benchmark_experiment
        benchmark_experiment = discover_benchmark_experiment(
            experiment_name=experiment_name,
            mlflow_client=mlflow_client,
            create_if_missing=True,
        )
    
    # Setup benchmark tracker
    benchmark_tracker = MLflowBenchmarkTracker(benchmark_experiment_name)
    
    # Run benchmarking workflow
    champions_to_benchmark = run_benchmarking_workflow(
        hpo_experiments=hpo_experiments,
        selection_config=selection_config,
        benchmark_config=benchmark_config,
        data_config=data_config,
        hpo_config=hpo_config,
        root_dir=ROOT_DIR,
        config_dir=CONFIG_DIR,
        experiment_name=experiment_name,
        mlflow_client=mlflow_client,
        benchmark_experiment=benchmark_experiment,
        benchmark_tracker=benchmark_tracker,
        backup_enabled=BACKUP_ENABLED,
        backup_to_drive=restore_from_drive if "restore_from_drive" in locals() else None,
        restore_from_drive=restore_from_drive if "restore_from_drive" in locals() else None,
        in_colab=IN_COLAB,
    )
    
    # Store benchmarked champions for checkpoint reuse in Step 7
    # Index by refit_run_id (primary) and (backbone, study_key_hash, trial_key_hash) (fallback)
    BENCHMARKED_CHAMPIONS_BY_REFIT = {}
    BENCHMARKED_CHAMPIONS_BY_KEYS = {}
    
    for backbone, champion_data in champions_to_benchmark.items():
        champion = champion_data.get("champion", {})
        refit_run_id = champion.get("refit_run_id")
        checkpoint_path = champion.get("checkpoint_path")
        
        if refit_run_id and checkpoint_path:
            # Primary index: refit_run_id (most reliable)
            BENCHMARKED_CHAMPIONS_BY_REFIT[refit_run_id] = {
                "checkpoint_path": Path(checkpoint_path),
                "backbone": backbone,
                "champion": champion,
            }
            
            # Fallback index: (backbone, study_key_hash, trial_key_hash)
            study_key_hash = champion.get("study_key_hash")
            trial_key_hash = champion.get("trial_key_hash")
            if study_key_hash and trial_key_hash:
                BENCHMARKED_CHAMPIONS_BY_KEYS[(backbone, study_key_hash, trial_key_hash)] = {
                    "checkpoint_path": Path(checkpoint_path),
                    "refit_run_id": refit_run_id,
                    "champion": champion,
                }
    
    if BENCHMARKED_CHAMPIONS_BY_REFIT:
        print(f"💾 Stored {len(BENCHMARKED_CHAMPIONS_BY_REFIT)} benchmarked champion(s) for checkpoint reuse")
else:
    print("⏭ Skipping benchmarking (RUN_BENCHMARKING=False).")
    print("   If benchmark runs don't exist, set RUN_BENCHMARKING=True or run benchmarking in notebook 01.")
    BENCHMARKED_CHAMPIONS_BY_REFIT = {}
    BENCHMARKED_CHAMPIONS_BY_KEYS = {}

2026-01-15 11:18:14,559 - evaluation.selection.workflows.benchmarking_workflow - INFO - Running benchmarking workflow on champions
2026-01-15 11:18:14,562 - evaluation.selection.workflows.benchmarking_workflow - INFO - Selecting champions for 2 backbone(s)
2026-01-15 11:18:14,822 - evaluation.selection.trial_finder - INFO - No runs found with stage='hpo_trial' for distilbert, trying legacy stage='hpo'
2026-01-15 11:18:15,031 - evaluation.selection.trial_finder - INFO - Found 3 runs with stage tag for distilbert (backbone=distilbert)
2026-01-15 11:18:15,032 - evaluation.selection.trial_finder - INFO - Filtered out 1 parent run(s) (only child/trial runs have metrics). 2 child runs remaining.
2026-01-15 11:18:15,033 - evaluation.selection.trial_finder - INFO - Grouped runs for distilbert: 1 v1 group(s), 0 v2 group(s)
2026-01-15 11:18:15,034 - evaluation.selection.trial_finder - INFO - Found 1 eligible group(s) for distilbert (0 skipped due to min_trials requirement)
2026-01-15 11:18:15,33

Loaded 2 test texts
Starting benchmark for checkpoint: /workspaces/resume-ner-azureml/outputs/artifacts/local/distilroberta/checkpoint_7eabeb17_681375f8
Loading tokenizer from /workspaces/resume-ner-azureml/outputs/artifacts/local/distilroberta/checkpoint_7eabeb17_681375f8...
Tokenizer loaded.
Loading model from /workspaces/resume-ner-azureml/outputs/artifacts/local/distilroberta/checkpoint_7eabeb17_681375f8...
Moving model to cpu...
Model loaded and set to eval mode.
Model ready on device: cpu

Benchmarking batch size 1...
  Running 10 warmup iterations, then 10 measurement iterations...
    Warmup: 10 iterations... 10/10 done.
    Measurement: 10 iterations... 10/10 done.
  Mean latency: 331.04 ms
  P95 latency: 459.54 ms
  Throughput: 3.02 docs/sec

Saving results to /workspaces/resume-ner-azureml/outputs/benchmarking/local/distilroberta/study-7eabeb17/trial-681375f8/bench-fcfe0227/benchmark.json...
Benchmark results saved to /workspaces/resume-ner-azureml/outputs/benchmarking/local

2026-01-15 11:18:46,234 - evaluation.benchmarking.utils - INFO - [Benchmark Run Name] Building run name: trial_id=681375f8734ec607, root_dir=/workspaces/resume-ner-azureml, config_dir=/workspaces/resume-ner-azureml/config
2026-01-15 11:18:46,274 - infrastructure.naming.mlflow.config - INFO - [Auto-Increment Config] Loading from config_dir=/workspaces/resume-ner-azureml/config, raw_auto_inc_config={'enabled': True, 'processes': {'hpo': True, 'benchmarking': True}, 'format': '{base}.{version}'}
2026-01-15 11:18:46,276 - infrastructure.naming.mlflow.config - INFO - [Auto-Increment Config] Validated config: {'enabled': True, 'processes': {'hpo': True, 'benchmarking': True}, 'format': '{base}.{version}'}, process_type=benchmarking
2026-01-15 11:18:46,277 - orchestration.jobs.tracking.index.version_counter - INFO - [Reserve Version] Starting reservation: counter_key=resume-ner:benchmarking:4972aae936a8ee0e92a512aea09ffff08589..., root_dir=/workspaces/resume-ner-azureml, config_dir=/workspace

🏃 View run local_distilroberta_benchmark_study-7eabeb17_trial-681375f8_bench-fcfe0227_1 at: https://germanywestcentral.api.azureml.ms/mlflow/v2.0/subscriptions/50c06ef8-627b-46d5-b779-d07c9b398f75/resourceGroups/resume_ner_2026-01-02-16-47-05/providers/Microsoft.MachineLearningServices/workspaces/resume-ner-ws/#/experiments/29716cbc-2f1e-485a-87be-3ef5c2f931dd/runs/ae21682e-5e74-49eb-9bbc-84c12c5c6a2a
🧪 View experiment at: https://germanywestcentral.api.azureml.ms/mlflow/v2.0/subscriptions/50c06ef8-627b-46d5-b779-d07c9b398f75/resourceGroups/resume_ner_2026-01-02-16-47-05/providers/Microsoft.MachineLearningServices/workspaces/resume-ner-ws/#/experiments/29716cbc-2f1e-485a-87be-3ef5c2f931dd


2026-01-15 11:18:50,515 - evaluation.benchmarking.orchestrator - INFO - Benchmark completed: /workspaces/resume-ner-azureml/outputs/benchmarking/local/distilroberta/study-7eabeb17/trial-681375f8/bench-fcfe0227/benchmark.json
2026-01-15 11:18:50,516 - evaluation.benchmarking.orchestrator - INFO - Benchmarking complete. 1/1 trials benchmarked.
2026-01-15 11:18:50,517 - evaluation.selection.workflows.benchmarking_workflow - INFO - Benchmarking complete for 1 champion(s)


💾 Stored 1 benchmarked champion(s) for checkpoint reuse


## Step 7: Best Model Selection

Query MLflow benchmark runs (created by `01_orchestrate_training_colab.ipynb` or Step 6 above using `evaluation.benchmarking.benchmark_best_trials`), join to training runs via grouping tags, and select the best model using normalized composite scoring.

**Note**: Benchmark runs must exist in MLflow before running this step. If no benchmark runs are found, either:
- Set `RUN_BENCHMARKING=True` in Step 6 above, or
- Go back to `01_orchestrate_training_colab.ipynb` and run the benchmarking step.


In [10]:
from evaluation.selection.workflows import run_selection_workflow
from training_exec import extract_lineage_from_best_model
from pathlib import Path

# Initialize dictionaries if not already created (e.g., if benchmarking was skipped)
if "BENCHMARKED_CHAMPIONS_BY_REFIT" not in globals():
    BENCHMARKED_CHAMPIONS_BY_REFIT = {}
if "BENCHMARKED_CHAMPIONS_BY_KEYS" not in globals():
    BENCHMARKED_CHAMPIONS_BY_KEYS = {}

# Run selection workflow
best_model, best_checkpoint_dir = run_selection_workflow(
    benchmark_experiment=benchmark_experiment,
    hpo_experiments=hpo_experiments,
    selection_config=selection_config,
    tags_config=tags_config,
    root_dir=ROOT_DIR,
    config_dir=CONFIG_DIR,
    experiment_name=experiment_name,
    acquisition_config=acquisition_config,
    platform=PLATFORM,
    restore_from_drive=restore_from_drive if "restore_from_drive" in locals() else None,
    drive_store=drive_store if "drive_store" in locals() else None,
    in_colab=IN_COLAB,
    benchmarked_champions_by_refit=BENCHMARKED_CHAMPIONS_BY_REFIT if "BENCHMARKED_CHAMPIONS_BY_REFIT" in globals() else None,
    benchmarked_champions_by_keys=BENCHMARKED_CHAMPIONS_BY_KEYS if "BENCHMARKED_CHAMPIONS_BY_KEYS" in globals() else None,
)

# Extract lineage from best model for final training
lineage = extract_lineage_from_best_model(best_model)

2026-01-15 11:21:27,839 - evaluation.selection.workflows.selection_workflow - INFO - Best Model Selection Mode: force_new
2026-01-15 11:21:27,845 - evaluation.selection.workflows.selection_workflow - INFO - Mode is 'force_new' - querying MLflow for fresh selection
2026-01-15 11:21:27,849 - evaluation.selection.mlflow_selection - INFO - Finding best model from MLflow
2026-01-15 11:21:27,849 - evaluation.selection.mlflow_selection - INFO -   Benchmark experiment: resume_ner_baseline-benchmark
2026-01-15 11:21:27,850 - evaluation.selection.mlflow_selection - INFO -   HPO experiments: 2
2026-01-15 11:21:27,851 - evaluation.selection.mlflow_selection - INFO -   Objective metric: macro-f1
2026-01-15 11:21:27,851 - evaluation.selection.mlflow_selection - INFO -   Composite weights: F1=0.70, Latency=0.30
2026-01-15 11:21:27,852 - evaluation.selection.mlflow_selection - INFO -   Latency aggregation: latest (from config file, applied when multiple benchmark runs exist with same benchmark_key)
20


✅ Best model selected:
   Run ID: 0f5c4ff8-c3c5-475c-9924-8d75d3016ab9
   Experiment: resume_ner_baseline-hpo-distilbert
   Backbone: distilbert
   F1 Score: 0.4530
   Latency: 175.13 ms
   Composite Score: 1.0000


In [11]:
# Check if selected run is already final training (skip retraining if so)
stage_tag = tags_config.key("process", "stage")
trained_on_full_data_tag = tags_config.key("training", "trained_on_full_data")

is_final_training = best_model["tags"].get(stage_tag) == "final_training"
used_full_data = (
    best_model["tags"].get(trained_on_full_data_tag) == "true" or
    best_model["params"].get("use_combined_data", "false").lower() == "true"
)

SKIP_FINAL_TRAINING = is_final_training and used_full_data

if SKIP_FINAL_TRAINING:
    final_checkpoint_dir = best_checkpoint_dir


## Step 8: Final Training

Run final training with best configuration if needed.


In [12]:
if not SKIP_FINAL_TRAINING:
    print("🔄 Starting final training with best configuration...")
    from training_exec import execute_final_training
    # Execute final training (uses final_training.yaml via load_final_training_config)
    # Will automatically reuse existing complete runs if run.mode: reuse_if_exists in final_training.yaml
    final_checkpoint_dir = execute_final_training(
        root_dir=ROOT_DIR,
        config_dir=CONFIG_DIR,
        best_model=best_model,
        experiment_config=experiment_config,
        lineage=lineage,
        training_experiment_name=training_experiment_name,
        platform=PLATFORM,
    )
else:
    print("✓ Skipping final training - using selected checkpoint")

# Backup final checkpoint to Google Drive if in Colab
if IN_COLAB and drive_store and final_checkpoint_dir:
    checkpoint_path = Path(final_checkpoint_dir).resolve()
    # Check if checkpoint is already in Drive
    if str(checkpoint_path).startswith("/content/drive"):
        print(f"\n✓ Final training checkpoint is already in Google Drive")
        print(f"  Drive path: {checkpoint_path}")
    else:
        try:
            print(f"\n📦 Backing up final training checkpoint to Google Drive...")
            result = drive_store.backup(checkpoint_path, expect="dir")
            if result.ok:
                print(f"✓ Successfully backed up final checkpoint to Google Drive")
                print(f"  Drive path: {result.dst}")
            else:
                print(f"⚠ Drive backup failed: {result.reason}")
                if result.error:
                    print(f"  Error: {result.error}")
        except Exception as e:
            print(f"⚠ Drive backup error: {e}")
            print(f"  Checkpoint is still available locally at: {final_checkpoint_dir}")

2026-01-15 11:21:30,789 - infrastructure.naming.mlflow.config - INFO - [Auto-Increment Config] Loading from config_dir=/workspaces/resume-ner-azureml/config, raw_auto_inc_config={'enabled': True, 'processes': {'hpo': True, 'benchmarking': True}, 'format': '{base}.{version}'}
2026-01-15 11:21:30,790 - infrastructure.naming.mlflow.config - INFO - [Auto-Increment Config] Validated config: {'enabled': True, 'processes': {'hpo': True, 'benchmarking': True}, 'format': '{base}.{version}'}, process_type=final_training


🔄 Starting final training with best configuration...
✓ Final training config loaded from final_training.yaml
✓ Output directory: /workspaces/resume-ner-azureml/outputs/final_training/local/distilbert/spec-1e6acb58_exec-d1c574bc/v1


2026-01-15 11:21:30,967 - training.execution.mlflow_setup - INFO - 🏃 View run local_distilbert_final_training_spec-1e6acb58_exec-d1c574bc_v1 at: https://germanywestcentral.api.azureml.ms/mlflow/v2.0/subscriptions/50c06ef8-627b-46d5-b779-d07c9b398f75/resourceGroups/resume_ner_2026-01-02-16-47-05/providers/Microsoft.MachineLearningServices/workspaces/resume-ner-ws/#/experiments/801daa4d-3a56-4952-a374-cf2c5a9c2846/runs/a0ccabb0-68ee-457a-8cfc-e31f33628db1
2026-01-15 11:21:30,968 - training.execution.mlflow_setup - INFO - Created MLflow run: local_distilbert_final_training_spec-1e6acb58_exec-d1c574bc_v1 (a0ccabb0-68e...)


✓ Created MLflow run: local_distilbert_final_training_spec-1e6acb58_exec-d1c574bc_v1 (a0ccabb0-68e...)
🔄 Running final training...


2026-01-15 11:22:11,547 - training.execution.subprocess_runner - INFO - 🏃 View run local_distilbert_final_training_spec-1e6acb58_exec-d1c574bc_v1 at: https://germanywestcentral.api.azureml.ms/mlflow/v2.0/subscriptions/50c06ef8-627b-46d5-b779-d07c9b398f75/resourceGroups/resume_ner_2026-01-02-16-47-05/providers/Microsoft.MachineLearningServices/workspaces/resume-ner-ws/#/experiments/801daa4d-3a56-4952-a374-cf2c5a9c2846/runs/a0ccabb0-68ee-457a-8cfc-e31f33628db1
🧪 View experiment at: https://germanywestcentral.api.azureml.ms/mlflow/v2.0/subscriptions/50c06ef8-627b-46d5-b779-d07c9b398f75/resourceGroups/resume_ner_2026-01-02-16-47-05/providers/Microsoft.MachineLearningServices/workspaces/resume-ner-ws/#/experiments/801daa4d-3a56-4952-a374-cf2c5a9c2846

2026-01-15 11:22:11,548 - training.execution.subprocess_runner - WARNING - 2026-01-15 11:21:33,956 - infrastructure.tracking.mlflow.compatibility - INFO - Applied Azure ML artifact compatibility patch
/opt/conda/envs/resume-ner-training/lib/py

✓ Saved metadata to: /workspaces/resume-ner-azureml/outputs/final_training/local/distilbert/spec-1e6acb58_exec-d1c574bc/v1/metadata.json
✓ Final training completed. Checkpoint: /workspaces/resume-ner-azureml/outputs/final_training/local/distilbert/spec-1e6acb58_exec-d1c574bc/v1/checkpoint
✓ MLflow run: a0ccabb0-68e...


## Step 9: Model Conversion & Optimization

Convert the final trained model to ONNX format with optimization.

In [13]:
# Extract parent training information for conversion
from common.shared.json_cache import load_json
from pathlib import Path

# Load metadata from final training output directory
final_training_metadata_path = final_checkpoint_dir.parent / "metadata.json"

if not final_training_metadata_path.exists():
    raise ValueError(
        f"Metadata file not found: {final_training_metadata_path}\n"
        "Please ensure final training completed successfully."
    )

metadata = load_json(final_training_metadata_path)
parent_spec_fp = metadata.get("spec_fp")
parent_exec_fp = metadata.get("exec_fp")
parent_training_run_id = metadata.get("mlflow", {}).get("run_id")

if not parent_spec_fp or not parent_exec_fp:
    raise ValueError(
        f"Missing required fingerprints in metadata: spec_fp={parent_spec_fp}, exec_fp={parent_exec_fp}\n"
        "Please ensure final training completed successfully."
    )

if parent_training_run_id:
    print(f"✓ Parent training: spec_fp={parent_spec_fp[:8]}..., exec_fp={parent_exec_fp[:8]}..., run_id={parent_training_run_id[:12]}...")
else:
    print(f"✓ Parent training: spec_fp={parent_spec_fp[:8]}..., exec_fp={parent_exec_fp[:8]}... (run_id not found)")

# Get parent training output directory (checkpoint parent)
parent_training_output_dir = final_checkpoint_dir.parent

print(f"\n🔄 Starting model conversion...")
from conversion import execute_conversion

# Execute conversion (uses conversion.yaml via load_conversion_config)
conversion_output_dir = execute_conversion(
    root_dir=ROOT_DIR,
    config_dir=CONFIG_DIR,
    parent_training_output_dir=parent_training_output_dir,
    parent_spec_fp=parent_spec_fp,
    parent_exec_fp=parent_exec_fp,
    experiment_config=experiment_config,
    conversion_experiment_name=conversion_experiment_name,
    platform=PLATFORM,
    parent_training_run_id=parent_training_run_id,  # May be None, that's OK
)

# Find ONNX model file (search recursively, as model may be in onnx_model/ subdirectory)
onnx_files = list(conversion_output_dir.rglob("*.onnx"))
if onnx_files:
    onnx_model_path = onnx_files[0]
    print(f"\n✓ Conversion completed successfully!")
    print(f"  ONNX model: {onnx_model_path}")
    print(f"  Model size: {onnx_model_path.stat().st_size / (1024 * 1024):.2f} MB")
else:
    print(f"\n⚠ Warning: No ONNX model file found in {conversion_output_dir} (searched recursively)")

# Backup conversion output to Google Drive if in Colab
if IN_COLAB and drive_store and conversion_output_dir:
    output_path = Path(conversion_output_dir).resolve()
    # Check if output is already in Drive
    if str(output_path).startswith("/content/drive"):
        print(f"\n✓ Conversion output is already in Google Drive")
        print(f"  Drive path: {output_path}")
    else:
        try:
            print(f"\n📦 Backing up conversion output to Google Drive...")
            result = drive_store.backup(output_path, expect="dir")
            if result.ok:
                print(f"✓ Successfully backed up conversion output to Google Drive")
                print(f"  Drive path: {result.dst}")
            else:
                print(f"⚠ Drive backup failed: {result.reason}")
                if result.error:
                    print(f"  Error: {result.error}")
        except Exception as e:
            print(f"⚠ Drive backup error: {e}")
            print(f"  Output is still available locally at: {conversion_output_dir}")


✓ Parent training: spec_fp=1e6acb58..., exec_fp=d1c574bc..., run_id=a0ccabb0-68e...

🔄 Starting model conversion...


/tmp/ipykernel_43989/2026750983.py:34: DeprecationWarning: conversion is deprecated, use deployment.conversion instead. This shim will be removed in 2 releases.
  from conversion import execute_conversion
[conversion.orchestration] Output directory: /workspaces/resume-ner-azureml/outputs/conversion/local/distilbert/spec-1e6acb58_exec-d1c574bc/v1/conv-e54f0100
2026-01-15 11:22:21,538 - infrastructure.naming.mlflow.config - INFO - [Auto-Increment Config] Loading from config_dir=/workspaces/resume-ner-azureml/config, raw_auto_inc_config={'enabled': True, 'processes': {'hpo': True, 'benchmarking': True}, 'format': '{base}.{version}'}
2026-01-15 11:22:21,539 - infrastructure.naming.mlflow.config - INFO - [Auto-Increment Config] Validated config: {'enabled': True, 'processes': {'hpo': True, 'benchmarking': True}, 'format': '{base}.{version}'}, process_type=conversion
[conversion.orchestration] Created MLflow run: local_distilbert_conversion_spec-1e6acb58_exec-d1c574bc_v1_conv-e54f0100 (02bb4


✓ Conversion completed successfully!
  ONNX model: /workspaces/resume-ner-azureml/outputs/conversion/local/distilbert/spec-1e6acb58_exec-d1c574bc/v1/conv-e54f0100/onnx_model/model.onnx
  Model size: 253.29 MB
